# Training and Health Analysis

## Training data
### We start by importing the data and cleaning it

I did a rough cleaning of the data in Power BI in order to more easily see what data to remove and change. Then for the more thorough cleaning I import all the data from csv-files into python with pandas.

In [18]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df1 = pd.read_csv('Lower A Clean.csv')
df2 = pd.read_csv('Lower B Clean.csv')
df3 = pd.read_csv('Upper A Clean.csv')
df4 = pd.read_csv('Upper B Clean.csv')
df5 = pd.read_csv('Upper Lower Maj Clean.csv')

frames = [df1,df2, df3, df4, df5]

df = pd.concat(frames)
df.rename(columns={"excercise": "exercise"}, inplace=True)
df.exercise = df.exercise.str.lower()
df.head()

,Date,exercise,set 1 reps,set 1 weight,set 2 reps,set 2 weight,set 3 reps,set 3 weight,set 4 reps,set 4 weight
0,2022-03-04 00:00:00,atg split squat,5.0,34.0,7,29.0,7.0,29.0,0,0.0
1,2022-02-15 00:00:00,atg split squat,6.0,29.0,6,29.0,5.0,29.0,0,0.0
2,2022-03-22 00:00:00,atg split squat,6.0,31.5,6,31.5,5.0,31.5,0,0.0
3,2022-03-31 00:00:00,atg split squat,6.0,31.5,6,31.5,6.0,31.5,0,0.0
4,2022-03-12 00:00:00,atg split squat,6.0,34.0,6,31.5,6.0,31.5,0,0.0


I first need to fix the dates so I can easily sort by year month and day.

In [19]:
df['date']=pd.to_datetime(df.Date)
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df = df.drop('Date', axis=1)  

df.head()

,exercise,set 1 reps,set 1 weight,set 2 reps,set 2 weight,set 3 reps,set 3 weight,set 4 reps,set 4 weight,date,year,month,day
0,atg split squat,5.0,34.0,7,29.0,7.0,29.0,0,0.0,2022-03-04,2022,3,4
1,atg split squat,6.0,29.0,6,29.0,5.0,29.0,0,0.0,2022-02-15,2022,2,15
2,atg split squat,6.0,31.5,6,31.5,5.0,31.5,0,0.0,2022-03-22,2022,3,22
3,atg split squat,6.0,31.5,6,31.5,6.0,31.5,0,0.0,2022-03-31,2022,3,31
4,atg split squat,6.0,34.0,6,31.5,6.0,31.5,0,0.0,2022-03-12,2022,3,12


I also want the reps and weights for all the sets in the same column so i need to do some manipulation here.

In [20]:
# melt the dataframe into a long format
melted_df = pd.melt(df, id_vars=['year', 'month', 'day', 'exercise'],
                    value_vars=['set 1 reps', 'set 2 reps', 'set 3 reps', 'set 4 reps',
                                'set 1 weight', 'set 2 weight', 'set 3 weight', 'set 4 weight'],
                    var_name='set', value_name='values')

# separate the 'set' column into 'set number' and 'metric' columns
melted_df[['set_name', 'set #', 'type']] = melted_df['set'].str.split(' ', expand=True)

# drop the original 'set' column
melted_df = melted_df.drop(columns=['set'])

# pivot the dataframe to get reps and weights in separate columns
pivoted_df = melted_df.pivot_table(index=['year', 'month', 'day', 'exercise', 'set #'],
                                   columns='type', values='values').reset_index()

pivoted_df.head()

type,year,month,day,exercise,set #,reps,weight
0,2021,12,3,barbell hacksquat,1,5.0,85.0
1,2021,12,3,barbell hacksquat,2,5.0,80.0
2,2021,12,3,barbell hacksquat,3,5.0,75.0
3,2021,12,3,barbell hacksquat,4,0.0,0.0
4,2021,12,3,rdls,1,18.0,60.0


Now it looks like I want it to, however, I want to remove the 'type' column header, which could be done by exporting the dataframe without index.

In [21]:
# removed type column by exporting
pivoted_df.to_csv('Trainingdata Final Clean.csv',index=False)

In [22]:
df_training = pd.read_csv('Trainingdata Final Clean.csv')
df_training = df_training.set_index(['year','month','day'])
df_training = df_training[df_training['reps'] != 0]

df_training.head(10)

exercise  set #  reps  weight
year month day                                        
2021 12    3    barbell hacksquat      1   5.0    85.0
           3    barbell hacksquat      2   5.0    80.0
           3    barbell hacksquat      3   5.0    75.0
           3                 rdls      1  18.0    60.0
           3                 rdls      2  16.0    60.0
           3                 rdls      3  15.0    60.0
           3      reverse nordics      1  12.0     0.0
           3      reverse nordics      2  12.0     0.0
           3      reverse nordics      3  12.0     0.0
           3     single leg hyper      1   8.0     0.0

We fix the index, and also remove all sets where I have done zero reps.

In [23]:
df_training = df_training[df_training['exercise'].str.contains('band') == False]

print('Number of different exercises: ',df_training['exercise'].nunique())

Number of different exercises:  77


I also removed all the exercises containing the word band, since I dont have a weight value for those.

In [24]:
exercise_rename_list = {
    'rdls':'romanian deadlifts',
    'regular dl':'deadlifts',
    'utfall':'lunges',
    'utfall?':'lunges',
    'bw lunges':'lunges',
    'bw hypers':'hyperextensions',
    'hyper extensions':'hyperextensions',
    'hypers':'hyperextensions',
    'hyperextension':'hyperextensions',
    'militärpress':'overhead press',
    'axelpress':'db shoulder press',
    'back squat':'squats',
    'backsquats':'squats',
    'bw pullups':'pull ups',
    'pullups':'pull ups',
    'pull ups*':'pull ups',
    'weighted pull ups':'pull ups',
    'weighted chin ups':'chin ups',
    'barbell benchpress':'bench press',
    'weighted push ups':'push ups',
    'weighted push up':'push ups',
    'sissy sqauts':'sissy squats',
    'biceps':'bicep curls',
    'bw atg split squats':'atg split squats',
    'atg split squat':'atg split squats',
    'oblique maskin':'obliques',
    'delt row':'inverted delt rows',
    'inverted delt row':'inverted delt rows',
    'rear delt row':'wide grip rows',
    'wide row':'wide grip rows',
    'trx rows':'inverted rows',
    'ring rows':'inverted rows',
    'inverted row':'inverted rows',
    'single leg machine curls':'hamstring curls',
    'leg curls':'hamstring curls',
    }
df_training['exercise'] = df_training['exercise'].replace(exercise_rename_list)
print("All exercises:\n",df_training.exercise.unique())
print("\nNumber of different exercises: ",df_training.exercise.nunique())

All exercises:
 ['barbell hacksquat' 'romanian deadlifts' 'reverse nordics'
 'single leg hyper' 'tibialis raise' 'bench press' 'bicep curls' 'dips'
 'incline db press' 'wide grip rows' 'side lateral' 'calves'
 'hamstring curls' 'leg press' 'obliques' 'quad extensions' 'sumo dl'
 'close grip row' 'overhead press' 'single leg hyper extensions'
 'straight arm pulldown' 'triceps' 'pull ups' 'decline push ups'
 'push ups' 'y raise' 'db shoulder press' 'hyperextensions'
 'barbell curls' 'atg split squats' 'nordic curls' 'lunges'
 'laying leg raise' 'machine press' 'chin ups' 'sissy squats'
 'decline press' 'inverted rows' 'lat pulldown' 'ring push ups'
 'inverted delt rows' 'deadlifts' 'beltsquat' 'squats'
 'bench press (pause)' 'tricep extensions' 'squat (pin)'
 'one arm lat pulldown' 'larsen press' 'good mornings'
 'atg heel elevated squat' 'db curls' 'ring curls' 'squat (pause)'
 'leg extensions' 'tricep pushdowns' 'pullover']

Number of different exercises:  57


In [25]:
print('\nNumber of sets of each exercise:\n',df_training['exercise'].value_counts())


Number of sets of each exercise:
 hamstring curls                147
sumo dl                        122
straight arm pulldown          102
side lateral                   102
push ups                       100
inverted rows                   93
pull ups                        92
squats                          88
reverse nordics                 76
romanian deadlifts              74
inverted delt rows              69
dips                            66
overhead press                  65
ring push ups                   64
atg split squats                60
decline push ups                59
tricep extensions               54
barbell hacksquat               54
barbell curls                   54
db shoulder press               52
one arm lat pulldown            48
db curls                        42
lat pulldown                    39
bench press                     39
hyperextensions                 39
wide grip rows                  36
squat (pause)                   33
chin ups            

In [26]:
df_training = df_training.groupby('exercise').filter(lambda x:len(x)>4)

print('\nNumber of sets of each exercise:\n',df_training['exercise'].value_counts())
print('\nlength: ',df_training.exercise.nunique())


Number of sets of each exercise:
 hamstring curls            147
sumo dl                    122
side lateral               102
straight arm pulldown      102
push ups                   100
inverted rows               93
pull ups                    92
squats                      88
reverse nordics             76
romanian deadlifts          74
inverted delt rows          69
dips                        66
overhead press              65
ring push ups               64
atg split squats            60
decline push ups            59
barbell curls               54
tricep extensions           54
barbell hacksquat           54
db shoulder press           52
one arm lat pulldown        48
db curls                    42
hyperextensions             39
lat pulldown                39
bench press                 39
wide grip rows              36
squat (pause)               33
chin ups                    32
sissy squats                27
decline press               27
squat (pin)                 24
atg 

## Weight data
### Start with cleaning

This time I can put the raw data directly in python because it is already pretty well structured.

In [27]:
df = pd.read_csv('weight.csv')
df = df.drop(columns=['Fat mass (kg)','Bone mass (kg)','Muscle mass (kg)', 'Hydration (kg)','Comments'],axis=1)
df = df.rename(columns={'Date':'date', 'Weight (kg)':'bodyweight'})

df.head()

,date,bodyweight
0,2023-02-24 09:35:18,79.80
1,2023-02-23 09:18:48,80.07
2,2023-02-22 10:32:51,80.74
3,2023-02-21 09:58:52,80.18
4,2023-02-20 08:57:37,80.27


In [28]:
df['date']=pd.to_datetime(df.date)
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df = df.drop('date', axis=1)
df = df.set_index(['year','month','day'])
df = df.iloc[::-1]

df.head()

bodyweight
year month day            
2021 10    21        76.72
           21        75.29
           22        74.80
           22        73.96
           22        74.37

We fixed the multi-index in the same way as the training data, but there are multiple entries for the same day.

We will take the mean for those and include only one entry per day.

In [29]:
df = df.groupby(['year','month','day']).mean()
df_weight = df
df_weight.head(10)

bodyweight
year month day            
2021 10    21      76.0050
           22      74.2250
           23      74.9100
           24      75.0050
           25      74.8225
           26      74.7950
           27      74.4400
           28      74.8900
           29      74.1200
           30      74.9200

Now it looks like how I want it.

# Sleep Data
## Cleaning

In [30]:
df = pd.read_csv('AutoSleep Raw.csv')
df.head()

,effektivitet,djup,ISO8601,kvalitet,sover,vaken
0,"99,6",03:21:41,2021-12-29 00:00:00,07:11:27,09:03:00,00:02:00
1,"100,0",03:05:15,2021-12-30 00:00:00,06:30:33,08:14:00,00:00:00
2,"93,3",02:50:11,2021-12-31 00:00:00,05:49:41,08:46:00,00:38:00
3,"94,2",03:04:08,2022-01-01 00:00:00,05:36:19,08:11:00,00:30:00
4,"96,3",02:20:04,2022-01-02 00:00:00,05:37:00,08:18:00,00:19:00


Start by removing all unnecessary columns and changing the headers

In [31]:
df = df.rename(columns={'ISO8601':'date','vaken':'time awake','sover':'time asleep','effektivitet':'effectivity','kvalitet':'quality sleep','djup':'deep sleep'})

df.head()

,effectivity,deep sleep,date,quality sleep,time asleep,time awake
0,"99,6",03:21:41,2021-12-29 00:00:00,07:11:27,09:03:00,00:02:00
1,"100,0",03:05:15,2021-12-30 00:00:00,06:30:33,08:14:00,00:00:00
2,"93,3",02:50:11,2021-12-31 00:00:00,05:49:41,08:46:00,00:38:00
3,"94,2",03:04:08,2022-01-01 00:00:00,05:36:19,08:11:00,00:30:00
4,"96,3",02:20:04,2022-01-02 00:00:00,05:37:00,08:18:00,00:19:00


In [32]:
df['date'] = pd.to_datetime(df['date'])
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df = df.drop('date', axis=1)
df = df.set_index(['year','month','day'])

df['deep sleep'] = pd.to_timedelta(df['deep sleep'])
df['quality sleep'] = pd.to_timedelta(df['quality sleep'])
df['time asleep'] = pd.to_timedelta(df['time asleep'])
df['time awake'] = pd.to_timedelta(df['time awake'])

df['deep sleep'] = pd.DataFrame(df['deep sleep']).apply(np.float32)
df['quality sleep'] = pd.DataFrame(df['quality sleep']).apply(np.float32)
df['time asleep'] = pd.DataFrame(df['time asleep']).apply(np.float32)
df['time awake'] = pd.DataFrame(df['time awake']).apply(np.float32)
df['effectivity'].str.replace(',','.').astype(float)


df.head()

effectivity    deep sleep  quality sleep   time asleep  \
year month day                                                          
2021 12    29         99,6  1.210100e+13   2.588700e+13  3.258000e+13   
           30        100,0  1.111500e+13   2.343300e+13  2.964000e+13   
           31         93,3  1.021100e+13   2.098100e+13  3.156000e+13   
2022 1     1          94,2  1.104800e+13   2.017900e+13  2.946000e+13   
           2          96,3  8.404000e+12   2.022000e+13  2.988000e+13   

                  time awake  
year month day                
2021 12    29   1.200000e+11  
           30   0.000000e+00  
           31   2.280000e+12  
2022 1     1    1.800000e+12  
           2    1.140000e+12

We can then join all the data tables on the index

In [33]:
df = df.join([df_training,df_weight],None,how='left')

df = df.reset_index(level=['year','month','day'])
df['date'] = pd.to_datetime(df[['year','month','day']])
df = df.drop(['year','month','day'], axis=1)
df.head()

,effectivity,deep sleep,quality sleep,time asleep,time awake,exercise,set #,reps,weight,bodyweight,date
0,"99,6",1.210100e+13,2.588700e+13,3.258000e+13,1.200000e+11,barbell hacksquat,1.0,6.0,90.0,76.205,2021-12-29
1,"99,6",1.210100e+13,2.588700e+13,3.258000e+13,1.200000e+11,barbell hacksquat,2.0,6.0,90.0,76.205,2021-12-29
2,"99,6",1.210100e+13,2.588700e+13,3.258000e+13,1.200000e+11,barbell hacksquat,3.0,5.0,90.0,76.205,2021-12-29
3,"99,6",1.210100e+13,2.588700e+13,3.258000e+13,1.200000e+11,barbell hacksquat,4.0,5.0,90.0,76.205,2021-12-29
4,"99,6",1.210100e+13,2.588700e+13,3.258000e+13,1.200000e+11,hyperextensions,1.0,36.0,0.0,76.205,2021-12-29


In [34]:
df = df.dropna(subset=['exercise'])

df.to_csv('Training and health data Clean.csv')